# 環境設定

## 安裝 PyTorch

In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

## 檢查 PyTorch 是否安裝成功

In [2]:
import torch

print(torch.__version__)

2.2.1+cu121


## 檢查 GPU 是否可用

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using GPU")
else:
  device = torch.device("cpu")
  print("Using CPU")

Using GPU


In [4]:
# 下載講師自製的 HappyML
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

In [5]:
# 載入必要套件
import pandas as pd
from sklearn.datasets import load_iris

import HappyML.preprocessor as pp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset

In [6]:
# 檢查是否有可用的 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Data Pre-processing

In [7]:
# Upload Dataset
Dataset_File = "Glass.csv"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [8]:
# Load Data
dataset = pp.dataset(file="Glass.csv")
print(dataset)

          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  Type
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0     1
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0     1
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0     1
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0     1
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0     1
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   ...
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0     7
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0     7
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0     7
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0     7
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0     7

[214 rows x 10 columns]


In [9]:
# 檢查是否有缺失資料, 再算每列有幾個, 再看看整個dataframe總共有幾個缺失資料
if sum(dataset.isnull().sum()) > 0:
  print('要補缺失資料')
else :
  print('不要補缺失資料')

不要補缺失資料


In [12]:
# 切分自變數與應變數 ← 不推薦混用 NDArray (x) 與 DataFrame (y)
# X = dataset.loc[:, dataset.columns[:-1]].values
# Y = pd.DataFrame(dataset.iloc[:, -1].values.reshape(-1, 1), columns=['Glass_Type'])

In [13]:
# 切分自變數與應變數 (by Robert)
import HappyML.preprocessor as pp
X, Y = pp.decomposition(dataset, x_columns=[i for i in range(9)], y_columns=[9])

In [15]:
# 可試著印出 X, Y, Y_name 驗證結果
X, Y

(          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe
 0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0
 1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0
 2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0
 3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0
 4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0
 ..       ...    ...   ...   ...    ...   ...   ...   ...  ...
 209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0
 210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0
 211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0
 212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0
 213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0
 
 [214 rows x 9 columns],
      Type
 0       1
 1       1
 2       1
 3       1
 4       1
 ..    ...
 209     7
 210     7
 211     7
 212     7
 213     7
 
 [214 rows x 1 columns])

In [ ]:
# 使用標籤編碼器，將應變數 Y 數位化
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# labelEncoder = LabelEncoder() # NO 換成 0, Yes 換成 1.
# Y = labelEncoder.fit_transform(Y).astype("float64") # 轉成小數

# 使用獨熱編碼器(one-hot encoding)，將自變數 X 數位化
# ary_dummies = pd.get_dummies(X[:, 0]).values
# X = np.concatenate((ary_dummies, X[:, 1:4]), axis=1).astype("float64")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
print(X)
print(Y)

          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0
..       ...    ...   ...   ...    ...   ...   ...   ...  ...
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0

[214 rows x 9 columns]
     Type
0       1
1       1
2       1
3       1
4       1
..    ...
209     7
210     7
211     7
212     7
213     7

[214 rows x 1 columns]


In [18]:
# 切分訓練集、測試集
X_train, X_test, Y_train, Y_test = pp.split_train_test(x_ary=X, y_ary=Y)

In [19]:
Y_train.shape

(160, 1)

In [20]:
print(X_train, X_test, Y_train, Y_test)

          RI     Na    Mg    Al     Si     K     Ca    Ba    Fe
39   1.52213  14.21  3.82  0.47  71.77  0.11   9.57  0.00  0.00
110  1.52664  11.23  0.00  0.77  73.21  0.00  14.68  0.00  0.00
178  1.51829  14.46  2.24  1.62  72.38  0.00   9.26  0.00  0.00
101  1.51730  12.35  2.72  1.63  72.87  0.70   9.23  0.00  0.00
148  1.51670  13.24  3.57  1.38  72.70  0.56   8.44  0.00  0.10
..       ...    ...   ...   ...    ...   ...    ...   ...   ...
124  1.52177  13.20  3.68  1.15  72.75  0.54   8.52  0.00  0.00
195  1.51545  14.14  0.00  2.68  73.39  0.08   9.07  0.61  0.05
116  1.51829  13.24  3.90  1.41  72.33  0.55   8.31  0.00  0.10
48   1.52223  13.21  3.77  0.79  71.99  0.13  10.02  0.00  0.00
200  1.51508  15.15  0.00  2.25  73.50  0.00   8.34  0.63  0.00

[160 rows x 9 columns]           RI     Na    Mg    Al     Si     K     Ca    Ba    Fe
6    1.51743  13.30  3.60  1.14  73.09  0.58   8.17  0.00  0.00
175  1.52119  12.97  0.33  1.51  73.39  0.13  11.27  0.00  0.28
154  1.51694  12

In [ ]:
# 特徵縮放 ← 切分訓練集用快樂版，這裡又換標準版。極度不推薦
# from sklearn.preprocessing import StandardScaler
#
# sc_X = StandardScaler().fit(X_train)
# X_train = sc_X.transform(X_train)
# X_test = sc_X.transform(X_test)

In [21]:
# 特徵縮放
X_train, X_test = pp.feature_scaling(fit_ary=X_train, transform_arys=(X_train, X_test))

In [23]:
# 可試著印出 X_train, X_test, Y_train, Y_test 驗證結果
print(X_train, X_test, end='\n')

           RI        Na        Mg        Al        Si         K        Ca  \
39   1.272025  0.874101  0.823395 -2.016887 -1.131299 -0.539504  0.477425   
110  2.791792 -2.611777 -1.739416 -1.410456  0.706344 -0.689817  4.320513   
178 -0.021967  1.166541 -0.236616  0.307763 -0.352853 -0.689817  0.244282   
101 -0.355574 -1.301648  0.085413  0.327978  0.272456  0.266720  0.221720   
148 -0.557760 -0.260564  0.655672 -0.177381  0.055512  0.075413 -0.372417   
..        ...       ...       ...       ...       ...       ...       ...   
124  1.150714 -0.307354  0.729470 -0.642311  0.119319  0.048083 -0.312251   
195 -0.978982  0.792218 -1.739416  2.450485  0.936049 -0.580499  0.101389   
116 -0.021967 -0.260564  0.877067 -0.116738 -0.416660  0.061748 -0.470186   
48   1.305723 -0.295656  0.789851 -1.370028 -0.850548 -0.512175  0.815857   
200 -1.103663  1.973674 -1.739416  1.581268  1.076424 -0.689817 -0.447624   

           Ba        Fe  
39  -0.387555 -0.572974  
110 -0.387555 -0.572974

In [25]:
# 將資料轉換成 PyTorch 張量
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)

In [26]:
# 可試著印出 X_train_tensor, X_test_tensor, Y_train_tensor, Y_test_tensor 驗證
print(X_train_tensor, X_test_tensor, Y_train_tensor, Y_test_tensor, end='\n')

tensor([[ 1.2720,  0.8741,  0.8234,  ...,  0.4774, -0.3876, -0.5730],
        [ 2.7918, -2.6118, -1.7394,  ...,  4.3205, -0.3876, -0.5730],
        [-0.0220,  1.1665, -0.2366,  ...,  0.2443, -0.3876, -0.5730],
        ...,
        [-0.0220, -0.2606,  0.8771,  ..., -0.4702, -0.3876,  0.4434],
        [ 1.3057, -0.2957,  0.7899,  ...,  0.8159, -0.3876, -0.5730],
        [-1.1037,  1.9737, -1.7394,  ..., -0.4476,  0.8156, -0.5730]]) tensor([[-3.1177e-01, -1.9038e-01,  6.7580e-01, -6.6253e-01,  5.5321e-01,
          1.0274e-01, -5.7548e-01, -3.8755e-01, -5.7297e-01],
        [ 9.5527e-01, -5.7640e-01, -1.5180e+00,  8.5406e-02,  9.3605e-01,
         -5.1217e-01,  1.7559e+00, -3.8755e-01,  2.2728e+00],
        [-4.7689e-01, -7.0507e-01,  6.6238e-01, -3.1888e-01, -5.9341e-02,
          1.4374e-01, -1.0919e-01, -3.8755e-01, -5.7297e-01],
        [-4.2634e-01, -5.4131e-01,  5.8858e-01,  6.5141e-01,  8.1035e-02,
          2.1206e-01, -5.6796e-01, -3.8755e-01, -5.7297e-01],
        [-6.3863e-01, 

In [27]:
print(X_train_tensor.shape)
print(X_test_tensor.shape)
print(Y_train_tensor.shape)
print(Y_test_tensor.shape)

torch.Size([160, 9])
torch.Size([54, 9])
torch.Size([160, 1])
torch.Size([54, 1])


# 模型定義

## 定義代表模型的類別

In [36]:
class GlassModel(nn.Module):

  # 定義神經網路每層架構
  def __init__(self):
    super(GlassModel, self).__init__()

    # 先定義每個神經層
    # self.fc1 = nn.Linear(181, 90)
    self.fc1 = nn.Linear(9, 90)
    self.fc2 = nn.Linear(90, 45)
    # self.fc3 = nn.Linear(45, 7)
    self.fc3 = nn.Linear(45, 8)

    # 接著初始化每個神經層的權重
    init.xavier_normal_(self.fc1.weight)
    init.xavier_normal_(self.fc2.weight)
    init.xavier_normal_(self.fc3.weight)

  # 定義輸入值 x 如何一路計算到輸出值（正向傳播）
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

## 定義優化器與損失函數

In [37]:
# 將模型實體化
model = GlassModel().to(device)

# 定義損失函數
criterion = nn.CrossEntropyLoss()

# 定義優化器
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 訓練模型

In [38]:
# 將資料集切割成數個 batch
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [39]:
# 開始訓練
#epochs = 33
epochs = 500

for epoch in range(epochs):
  # 將模型設定為訓練模式
  model.train()

  # 儲存猜對的數量 ＆ 完整數量
  correct = 0
  total = 0

  # 取出一個 Batch 開始訓練
  for X_batch, Y_batch in train_loader:
    # 將 Batch 的資料轉換到 GPU 上
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    # 先把上一個 Batch 的梯度歸零
    optimizer.zero_grad()

    # 計算輸出值
    Y_pred = model(X_batch)
    # 計算損失值
    loss = criterion(Y_pred, Y_batch.squeeze())
    # 根據損失值求微分找損失極小的權重（反向傳播）
    loss.backward()
    # 將求出來的權重實際更新上去
    optimizer.step()

    # 計算第 1 軸的每一列，最大值之索引為何
    _, predicted = torch.max(Y_pred.data, 1)
    # 將這一批次有幾筆資料加入到 total 中
    total += Y_batch.size(0)
    # 計算猜對的數量（.item() 會協助取得純量）
    correct += (predicted == Y_batch.squeeze()).sum().item()

  # 計算每個 epoch 的準確率
  accuracy = correct / total
  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Acc: {accuracy:.4f}')

Epoch 1/500, Loss: 1.9729, Acc: 0.1500
Epoch 2/500, Loss: 1.9365, Acc: 0.1938
Epoch 3/500, Loss: 2.0091, Acc: 0.2062
Epoch 4/500, Loss: 2.0229, Acc: 0.2188
Epoch 5/500, Loss: 1.9891, Acc: 0.2687
Epoch 6/500, Loss: 2.0016, Acc: 0.2625
Epoch 7/500, Loss: 1.9357, Acc: 0.3000
Epoch 8/500, Loss: 1.8582, Acc: 0.3125
Epoch 9/500, Loss: 1.8927, Acc: 0.3187
Epoch 10/500, Loss: 1.9619, Acc: 0.3312
Epoch 11/500, Loss: 1.9359, Acc: 0.3375
Epoch 12/500, Loss: 1.8694, Acc: 0.3375
Epoch 13/500, Loss: 1.9382, Acc: 0.3438
Epoch 14/500, Loss: 1.7832, Acc: 0.3563
Epoch 15/500, Loss: 1.8572, Acc: 0.3625
Epoch 16/500, Loss: 1.8239, Acc: 0.3875
Epoch 17/500, Loss: 1.8657, Acc: 0.4062
Epoch 18/500, Loss: 1.7478, Acc: 0.4188
Epoch 19/500, Loss: 1.8332, Acc: 0.4188
Epoch 20/500, Loss: 1.8359, Acc: 0.4250
Epoch 21/500, Loss: 1.8099, Acc: 0.4500
Epoch 22/500, Loss: 1.7273, Acc: 0.4500
Epoch 23/500, Loss: 1.7516, Acc: 0.4562
Epoch 24/500, Loss: 1.7511, Acc: 0.4688
Epoch 25/500, Loss: 1.7085, Acc: 0.4750
Epoch 26/

# 評估模型

In [40]:
# 將模型切換為評估模式
model.eval()

# 關閉 PyTorch 的梯度計算機制（Evaluation 時期不需要它）
with torch.no_grad():

  # 儲存猜對的數量 ＆ 完整數量
  correct = 0
  total = 0

  # 取出測試集的一個批次，開始測試
  for X_batch, Y_batch in test_loader:
    # 將 Batch 的資料轉換到 GPU 上
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    # 計算輸出值
    Y_pred = model(X_batch)

    # 找到每一列預測機率最高的數值與其索引，即模型的預測類別。
    _, predicted = torch.max(Y_pred.data, 1)
    # 將這一批次有幾筆資料加入到 total 中
    total += Y_batch.size(0)
    # 計算猜對的數量（.item() 會協助取得純量）
    correct += (predicted == Y_batch.squeeze()).sum().item()

  # 計算每個 epoch 的準確率
  print(f'Test Accuracy: {correct / total:.4f}')

Test Accuracy: 0.6111


# 預測答案

In [42]:
import numpy as np

# 將模型切換為評估模式
model.eval()

# 儲存預測結果
Y_prediction = []

# 關閉 PyTorch 的梯度計算機制（Evaluation 時期不需要它）
with torch.no_grad():

  # 取出測試集的一個批次，開始測試
  for X_batch, Y_batch in test_loader:
    # 將 Batch 的資料轉換到 GPU 上
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

    # 計算輸出值
    Y_pred = model(X_batch)

    # 找到每一列預測機率最高的數值與其索引，即模型的預測類別。
    _, predicted = torch.max(Y_pred.data, 1)

    # 將預測結果添加到列表中
    Y_prediction.extend(np.float64(predicted.cpu()))

Y_prediction = np.array(Y_prediction)

In [43]:
print(type(Y_prediction[0]))

<class 'numpy.float64'>


In [44]:
print(Y_test, end='\n')
# 輸出預測結果
print(Y_prediction, end='\n')

     Type
6       1
175     5
154     3
143     2
82      2
144     2
111     2
109     2
81      2
24      1
142     2
127     2
40      1
26      1
107     2
54      1
167     5
119     2
104     2
19      1
10      1
58      1
52      1
155     3
60      1
18      1
157     3
165     5
132     2
163     5
3       1
149     3
209     7
50      1
84      2
146     3
9       1
181     6
90      2
213     7
133     2
12      1
96      2
23      1
77      2
80      2
5       1
95      2
137     2
78      2
75      2
7       1
38      1
66      1
[1. 2. 1. 2. 1. 1. 2. 6. 2. 1. 2. 2. 1. 2. 2. 1. 5. 2. 2. 2. 2. 1. 1. 1.
 1. 1. 1. 5. 1. 7. 1. 1. 7. 1. 2. 1. 2. 6. 1. 7. 2. 2. 1. 1. 2. 2. 2. 1.
 2. 3. 2. 1. 1. 1.]
